### Predicting formation enthalpies for solid solutions of Lanthanides Orthophosphates


In [1]:
import os
import sys
import inspect

print(sys.version)

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
grandparentdir = os.path.dirname(parentdir)

sys.path.insert(0, grandparentdir) 

3.9.6 (default, Nov 10 2023, 13:38:27) 
[Clang 15.0.0 (clang-1500.1.0.2.5)]


In [2]:
import read_data
import featureSpan
import lasso
import Utils
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import scipy.optimize
import pandas as pd
import seaborn as sns
import scipy.stats as ss
from sklearn.linear_model import Lasso
from itertools import combinations, product
import itertools
import math
import pandas as pd

## Xenotime

#### Load data from files

In [3]:
list1 = read_data.readData("../../data/DATA_HE_xenotime.dat")
print("Shape of List1 is {}".format(list1.shape))
print(list1)

Shape of List1 is (525, 4)
[[5.70000000e+01 5.80000000e+01 7.50000000e-01 2.31598225e-01]
 [5.70000000e+01 5.80000000e+01 6.25000000e-01 2.78958493e-01]
 [5.70000000e+01 5.80000000e+01 5.00000000e-01 2.97583757e-01]
 ...
 [7.00000000e+01 7.10000000e+01 5.00000000e-01 7.77590152e-02]
 [7.00000000e+01 7.10000000e+01 3.75000000e-01 7.09266168e-02]
 [7.00000000e+01 7.10000000e+01 2.50000000e-01 5.91211365e-02]]


In [4]:
list2 = read_data.readCSVData_simplified("../../data/Data_Ln-xenotime.csv", material="xenotime", R=False)
print("Shape of List2 is {}".format(list2.shape))
print((list2))

Shape of List2 is (15, 2)
[[ 91.7         83.16086967]
 [ 99.73        81.11579575]
 [112.27        79.31176899]
 [120.19        77.89975607]
 [127.26        76.63526873]
 [137.16        75.42961529]
 [143.83        74.36126731]
 [149.42        73.33463097]
 [156.33        72.43898327]
 [163.          71.49567572]
 [169.12        70.75467386]
 [177.09        69.85074728]
 [183.38        68.92826151]
 [188.35        68.35674618]
 [194.41        67.43658042]]


#### Generated Elemental Training/Testing sets from loaded data

In [5]:
listX, listY = featureSpan.generateFeatures(list1, list2)
X = np.asarray(listX)
Y = np.asarray(listY)
print(X.shape)
print(Y.shape)

elemental_list = ["m", "1/m", "m^2", "(1/m)^2", "(1-m)", "1/(1-m)", "(1-m)^2", "(1/(1-m))^2", "[AD:Young]", "[AD:Volume]", "1/[AD:Young]", "1/[AD:Volume]", "[AD:Young]^2", "[AD:Volume]^2", "(1/[AD:Young])^2", "(1/[AD:Volume])^2", "[AM:Young]", "[AM:Volume]", "1/[AM:Young]", "1/[AM:Volume]","[AM:Young]^2", "[AM:Volume]^2", "(1/[AM:Young])^2", "(1/[AM:Volume]])^2"]
print(len(elemental_list))
print(elemental_list)
m, n = X.shape

elemental_features = []

for i in elemental_list:
    elemental_features.append("("+i+")")
        
print("The elemental features are: \n {}".format(elemental_features))

(525, 24)
(525,)
24
['m', '1/m', 'm^2', '(1/m)^2', '(1-m)', '1/(1-m)', '(1-m)^2', '(1/(1-m))^2', '[AD:Young]', '[AD:Volume]', '1/[AD:Young]', '1/[AD:Volume]', '[AD:Young]^2', '[AD:Volume]^2', '(1/[AD:Young])^2', '(1/[AD:Volume])^2', '[AM:Young]', '[AM:Volume]', '1/[AM:Young]', '1/[AM:Volume]', '[AM:Young]^2', '[AM:Volume]^2', '(1/[AM:Young])^2', '(1/[AM:Volume]])^2']
The elemental features are: 
 ['(m)', '(1/m)', '(m^2)', '((1/m)^2)', '((1-m))', '(1/(1-m))', '((1-m)^2)', '((1/(1-m))^2)', '([AD:Young])', '([AD:Volume])', '(1/[AD:Young])', '(1/[AD:Volume])', '([AD:Young]^2)', '([AD:Volume]^2)', '((1/[AD:Young])^2)', '((1/[AD:Volume])^2)', '([AM:Young])', '([AM:Volume])', '(1/[AM:Young])', '(1/[AM:Volume])', '([AM:Young]^2)', '([AM:Volume]^2)', '((1/[AM:Young])^2)', '((1/[AM:Volume]])^2)']


In [6]:
dfX = pd.DataFrame(data=X, columns=elemental_features)

new_features=[]
new_columns = []

for i in range(n):
    for j in range(i):
        new_features.append(elemental_features[i]+"*"+elemental_features[j])
        new_columns.append(dfX[elemental_features[i]].values * dfX[elemental_features[j]].values)


for i in range(n):
    for j in range(i):
        for k in range(j):
            new_features.append(elemental_features[i]+"*"+elemental_features[j]+"*"+elemental_features[k])
            new_columns.append(dfX[elemental_features[i]] * dfX[elemental_features[j]] * dfX[elemental_features[k]])
            

#new_columns = np.reshape(np.asarray(new_columns))
            
new_columns = np.asarray(new_columns)
dfX = pd.concat(
    [
        dfX,
        pd.DataFrame(
            new_columns.T, 
            index=dfX.index, 
            columns=new_features
        )
    ], axis=1
)

dfX.head()

dfX.std() == 0
dfX=dfX.loc[:, dfX.std() > 0]
dfX.head()

,(m),(1/m),(m^2),((1/m)^2),((1-m)),(1/(1-m)),((1-m)^2),((1/(1-m))^2),([AD:Young]),([AD:Volume]),...,((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*([AD:Young]^2),((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*([AD:Volume]^2),((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*((1/[AD:Young])^2),((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*((1/[AD:Volume])^2),((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*([AM:Young]),((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*([AM:Volume]),((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*(1/[AM:Young]),((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*(1/[AM:Volume]),((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*([AM:Young]^2),((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*([AM:Volume]^2)
0,0.750,1.333333,0.562500,1.777778,0.250,4.000000,0.062500,16.000000,4.015,1.022537,...,2.608070e-07,1.691633e-08,1.003638e-09,1.547356e-08,0.000002,0.000001,1.690317e-10,1.969710e-10,0.000148,0.000109
1,0.625,1.600000,0.390625,2.560000,0.375,2.666667,0.140625,7.111111,4.015,1.022537,...,2.608070e-07,1.691633e-08,1.003638e-09,1.547356e-08,0.000002,0.000001,1.690317e-10,1.969710e-10,0.000148,0.000109
2,0.500,2.000000,0.250000,4.000000,0.500,2.000000,0.250000,4.000000,4.015,1.022537,...,2.608070e-07,1.691633e-08,1.003638e-09,1.547356e-08,0.000002,0.000001,1.690317e-10,1.969710e-10,0.000148,0.000109
3,0.375,2.666667,0.140625,7.111111,0.625,1.600000,0.390625,2.560000,4.015,1.022537,...,2.608070e-07,1.691633e-08,1.003638e-09,1.547356e-08,0.000002,0.000001,1.690317e-10,1.969710e-10,0.000148,0.000109
4,0.250,4.000000,0.062500,16.000000,0.750,1.333333,0.562500,1.777778,4.015,1.022537,...,2.608070e-07,1.691633e-08,1.003638e-09,1.547356e-08,0.000002,0.000001,1.690317e-10,1.969710e-10,0.000148,0.000109


In [7]:
print(len(dfX.columns.values))
np.array(dfX.columns.values)

2320


array(['(m)', '(1/m)', '(m^2)', ...,
       '((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*(1/[AM:Volume])',
       '((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*([AM:Young]^2)',
       '((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*([AM:Volume]^2)'],
      dtype=object)

fitting of Lasso with a given $\lambda$

In [8]:
def LassoFit(lmb, X, Y, max_iter=100000, standardization = True):
    
    scaler = StandardScaler()
    scaler.fit(X)
    X_standardized = scaler.transform(X)
    lasso =  Lasso(alpha=lmb, max_iter=max_iter)
    lasso.fit(X_standardized, Y.copy())
    coef =  lasso.coef_
    selected_indices = coef.nonzero()[0]
    selected_features = np.array(dfX.columns.values)[selected_indices]
    Y_predict = lasso.predict(X_standardized)
    MAE, MSE, ME = Utils.compute_error(Y.copy(), Y_predict)
        
    return coef, selected_indices, selected_features, MAE, MSE, ME

LassoFit(0.01, dfX, Y)

(array([ 0., -0.,  0., ..., -0., -0., -0.]),
 array([  56,   64,  102,  108,  153,  204,  223,  286,  388,  533,  555,
         588,  593,  597,  599,  606,  613,  621,  944, 2155, 2251]),
 array(['([AD:Volume])*(m)', '([AD:Volume])*([AD:Young])',
        '([AD:Volume]^2)*((1-m))', '([AD:Volume]^2)*(1/[AD:Young])',
        '([AM:Young])*([AD:Volume]^2)', '(1/[AM:Volume])*([AD:Volume]^2)',
        '([AM:Young]^2)*([AD:Volume]^2)',
        '((1/[AM:Volume]])^2)*([AD:Volume]^2)',
        '([AD:Volume])*((1-m))*(m^2)', '([AD:Young]^2)*((1-m)^2)*(m^2)',
        '([AD:Young]^2)*([AD:Volume])*((1/m)^2)',
        '([AD:Volume]^2)*((1-m))*(m)', '([AD:Volume]^2)*(1/(1-m))*(1/m)',
        '([AD:Volume]^2)*((1-m)^2)*(m)', '([AD:Volume]^2)*((1-m)^2)*(m^2)',
        '([AD:Volume]^2)*((1/(1-m))^2)*((1/m)^2)',
        '([AD:Volume]^2)*([AD:Young])*((1/m)^2)',
        '([AD:Volume]^2)*([AD:Volume])*((1/m)^2)',
        '([AM:Young])*([AD:Volume]^2)*(1/[AD:Young])',
        '((1/[AM:Volume]])^2)*([AD:Vol

#### Analyze the behavior of Lasso with the decrease of $\lambda$

In [9]:
#### Define a function which fits Lasso to have no more nonzero coefficients than a given threshold 
def LassoSelect(X, Y, min, max, step, threshold, standardization = True):
    
    scaler = StandardScaler()
    scaler.fit(X)
    X_standardized = scaler.transform(X)
        
    found = False
    for lmbda in np.arange (min, max, step):
        coef, selected_indices, selected_features, MAE, MSE, ME = LassoFit(lmbda, X.copy(), Y.copy())
        if len(selected_indices) <= threshold:
            found = True
            break
    
    if found:
        print("FOUND with threshold: {}".format(threshold))
        print("Lambda: {}, nnz: {}, MAE: {}, MSE: {}, MAPE: {}".format(lmbda, len(selected_indices), MAE, MSE, ME))
            
    else:
        print("NOT FOUND with threshold: {}".format(threshold))
        print("Closest are: ")
        print("Lambda: {}, nnz: {}, MAE: {}, MSE: {}, MAPE: {}".format(lmbda, len(selected_indices), MAE, MSE, ME))
     
    X_reduced = X[selected_features]
    
    return X_reduced


X_reduced = LassoSelect(dfX, Y, 0.001, 0.101, 0.005, 10)
X_reduced.head()

FOUND with threshold: 10
Lambda: 0.061, nnz: 10, MAE: 0.06887241405275384, MSE: 0.010234855663507705, MAPE: 0.14336058598699092


,([AM:Young])*([AD:Volume]^2),([AM:Young]^2)*([AD:Volume]^2),([AD:Young]^2)*((1-m))*(m),([AD:Young]^2)*((1-m)^2)*(m^2),([AD:Volume]^2)*((1-m))*(m),([AD:Volume]^2)*((1-m)^2)*(m),([AD:Volume]^2)*((1-m)^2)*(m^2),([AM:Young])*([AD:Volume]^2)*((1-m)),([AM:Young])*([AD:Volume]^2)*(1/[AD:Young]),((1/[AM:Volume]])^2)*([AM:Young])*([AD:Volume]^2)
0,100.077865,9578.952833,3.022542,0.566727,0.196047,0.049012,0.036759,25.019466,24.925994,0.014834
1,100.077865,9578.952833,3.778178,0.885510,0.245058,0.091897,0.057436,37.529199,24.925994,0.014834
2,100.077865,9578.952833,4.030056,1.007514,0.261395,0.130698,0.065349,50.038932,24.925994,0.014834
3,100.077865,9578.952833,3.778178,0.885510,0.245058,0.153161,0.057436,62.548666,24.925994,0.014834
4,100.077865,9578.952833,3.022542,0.566727,0.196047,0.147035,0.036759,75.058399,24.925994,0.014834


#### Lasso with $\ell_0$

In [10]:
def LassoL0(X, Y, nnz):    
    nr, nc = X.shape
    X = np.column_stack((X, np.ones(nr)))
    se_min = np.inner(Y, Y)
    coef_min, permu_min = None, None
    for permu in combinations(range(nc), nnz):
        X_ls = X[:, permu + (-1,)]
        coef, se, __1, __2 = np.linalg.lstsq(X_ls, Y, rcond=-1)
        try:
            if se[0] < se_min: 
                se_min = se[0]
                coef_min, permu_min = coef, permu
        except:
            pass
        
    return coef_min, permu_min

In [11]:
def LassoL0Fit(X_reduced, Y, nnz, log=True):
    
    scaler = StandardScaler()
    scaler.fit(X_reduced)
    X_std = scaler.transform(X_reduced)
    
    nr, nc = X_reduced.shape
   
    coefficients, selected_indices = LassoL0(X_std, Y, nnz)
   
    coefficients = np.array(coefficients)
    selected_indices = np.array(selected_indices)
    feature_reduced = np.array(X_reduced.columns.values)
    feature_list_selected = feature_reduced[selected_indices]
    
    X_selected = X_reduced[feature_list_selected]

    mean_selected = X_selected.mean()
    std_selected = X_selected.std()

    
    if log:
        print("Lasso: selected coefficients are: {}".format(coefficients))
        print("Lasso: selected features are: {}".format(feature_list_selected))
        
    #-mean/std
    mean_std = []
    for i in range(len(selected_indices)):
        mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
 
    sum_mean_std = sum(mean_std)

    for i in range(len(selected_indices)):
        coefficients[i] = coefficients[i] / std_selected[i]

    
    coefficients[len(selected_indices)] -= sum_mean_std
    
    function = str(coefficients[0])+" * "+feature_list_selected[0]
    
    for i in range(1, len(selected_indices)):
        if coefficients[i] >= 0:
            function += " + " + str(coefficients[i])+" * "+feature_list_selected[i]
        else:
            function += " - " + str(abs(coefficients[i]))+" * "+feature_list_selected[i]

    
    if coefficients[len(selected_indices)] >= 0:
        function += " + " + str(coefficients[len(selected_indices)])
    else:
        function += " - " + str(abs(coefficients[len(selected_indices)]))
    
    if log:
        print("Constructed function is: {}".format(function))

    X_selected = np.column_stack((X_selected, np.ones(X_selected.shape[0])))
    Y_predict = X_selected[:,0] * coefficients[0]

    for i in range(1,len(selected_indices)+1):
        Y_predict = Y_predict + X_selected[:,i] * coefficients[i]
    
    if log:
        Utils.print_error(Y.copy(),Y_predict,"Lasso L0: {} coef".format(nnz))
    
    return Y_predict, coefficients, selected_indices

In [12]:
LassoL0Fit(X_reduced, Y.copy(), 1);

Lasso: selected coefficients are: [3.6874823 3.1736172]
Lasso: selected features are: ['([AD:Volume]^2)*((1-m))*(m)']
Constructed function is: 1.2264765333867818 * ([AD:Volume]^2)*((1-m))*(m) + 0.06308556014158206
Lasso L0: 1 coef
Mean absolute error: 0.16999152916334842
Mean squared error: 0.06885010753784614
Mean absolute percentage error: 0.2021633881048294


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_20919/1248470983.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_20919/1248470983.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]


In [13]:
LassoL0Fit(X_reduced, Y.copy(), 2);

Lasso: selected coefficients are: [1.96095066 1.77770019 3.1736172 ]
Lasso: selected features are: ['([AM:Young])*([AD:Volume]^2)' '([AD:Volume]^2)*((1-m)^2)*(m^2)']
Constructed function is: 0.0010260376673477058 * ([AM:Young])*([AD:Volume]^2) + 2.579989589338383 * ([AD:Volume]^2)*((1-m)^2)*(m^2) + 0.029643792688973658
Lasso L0: 2 coef
Mean absolute error: 0.0850890795231624
Mean squared error: 0.025208161111726086
Mean absolute percentage error: 0.1033217914826727


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_20919/1248470983.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_20919/1248470983.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]


In [14]:
LassoL0Fit(X_reduced, Y.copy(), 3);

Lasso: selected coefficients are: [1.67047461 1.77770019 0.32061262 3.1736172 ]
Lasso: selected features are: ['([AM:Young])*([AD:Volume]^2)' '([AD:Volume]^2)*((1-m)^2)*(m^2)'
 '([AM:Young])*([AD:Volume]^2)*((1-m))']
Constructed function is: 0.000874050483930846 * ([AM:Young])*([AD:Volume]^2) + 2.5799895893383025 * ([AD:Volume]^2)*((1-m)^2)*(m^2) + 0.0003039743668337763 * ([AM:Young])*([AD:Volume]^2)*((1-m)) + 0.0296437926889781
Lasso L0: 3 coef
Mean absolute error: 0.05475384148280947
Mean squared error: 0.0067920586196732035
Mean absolute percentage error: 0.10085054801459159


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_20919/1248470983.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_20919/1248470983.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]


In [15]:
LassoL0Fit(X_reduced, Y.copy(), 4);

Lasso: selected coefficients are: [-2.3438992   1.77581914  0.31904368  4.01355655  3.1736172 ]
Lasso: selected features are: ['([AM:Young]^2)*([AD:Volume]^2)' '([AD:Volume]^2)*((1-m)^2)*(m^2)'
 '([AM:Young])*([AD:Volume]^2)*((1-m))'
 '((1/[AM:Volume]])^2)*([AM:Young])*([AD:Volume]^2)']
Constructed function is: -8.613993668738834e-06 * ([AM:Young]^2)*([AD:Volume]^2) + 2.5772596020997334 * ([AD:Volume]^2)*((1-m)^2)*(m^2) + 0.00030248684158813287 * ([AM:Young])*([AD:Volume]^2)*((1-m)) + 11.86813708430403 * ((1/[AM:Volume]])^2)*([AM:Young])*([AD:Volume]^2) + 0.0031437652015791073
Lasso L0: 4 coef
Mean absolute error: 0.04498602823024534
Mean squared error: 0.004668711035258494
Mean absolute percentage error: 0.04124155444208587


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_20919/1248470983.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_20919/1248470983.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]


In [16]:
LassoL0Fit(X_reduced, Y.copy(), 5);

Lasso: selected coefficients are: [-5.36143064 -1.72159004  2.7025569   0.32180538  7.83059915  3.1736172 ]
Lasso: selected features are: ['([AM:Young]^2)*([AD:Volume]^2)' '([AD:Volume]^2)*((1-m))*(m)'
 '([AD:Volume]^2)*((1-m)^2)*(m^2)' '([AM:Young])*([AD:Volume]^2)*((1-m))'
 '((1/[AM:Volume]])^2)*([AM:Young])*([AD:Volume]^2)']
Constructed function is: -1.9703632951709814e-05 * ([AM:Young]^2)*([AD:Volume]^2) - 0.572610148147516 * ([AD:Volume]^2)*((1-m))*(m) + 3.9222410563153667 * ([AD:Volume]^2)*((1-m)^2)*(m^2) + 0.0003051052234372737 * ([AM:Young])*([AD:Volume]^2)*((1-m)) + 23.155179952420955 * ((1/[AM:Volume]])^2)*([AM:Young])*([AD:Volume]^2) - 0.015498086621307916
Lasso L0: 5 coef
Mean absolute error: 0.04089681872292345
Mean squared error: 0.0037589932608058977
Mean absolute percentage error: 0.04356308494500676


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_20919/1248470983.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_20919/1248470983.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]
